## VAE Hawkes Process Estimation - Tutorial

In [142]:
import os
import sys

import numpy as np
import pandas as pd
import Hawkes as hk
from matplotlib import pyplot as plt

from PREPROCESSING.hawkes import hawkes_simulations, hawkes_simulation
from PREPROCESSING.hyperparameters import hyper_params_simulation
from PREPROCESSING.discretisation import discretise
import VARIABLES.variables as var

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Preprocessing

In [144]:
# Training dataset generation

# Intensity Decay Parameter (β) = U(p = 1, q = 3)
# Branching Ratio (η) = U(a = 0.05, b = 0.8)
# Expected Activity (E) = 500
# Time Horizon (T) = 100
# Interval Length (∆) = 1
# Training processes = 100_000

# Hawkes process hyper-parameters generation
train_params, train_alpha, train_beta, train_mu = hyper_params_simulation(filename="train_hawkes_hyperparams.csv")

# Hawkes processes simulations
train_simulated_events_seqs = hawkes_simulations(train_mu, train_alpha, train_beta, filename='train_hawkes_simulations.csv')

# Discrétiser les processus de Hawkes
#train_discret_simulated_events_seqs = discretise(train_simulated_events_seqs, filename='train_binned_hawkes_simulations.csv')

In [ ]:
# Validating dataset generation

# Intensity Decay Parameter (β) = U(p = 1, q = 3)
# Branching Ratio (η) = U(a = 0.05, b = 0.8)
# Expected Activity (E) = 500
# Time Horizon (T) = 100
# Interval Length (∆) = 1
# Training processes = 30_000

# Hawkes process hyper-parameters generation
val_params, val_alpha, val_beta, val_mu = hyper_params_simulation(filename="val_hawkes_hyperparams.csv")
# Hawkes processes simulations
val_simulated_events_seqs = hawkes_simulations(val_mu, val_alpha, val_beta, filename='val_hawkes_simulations.csv')
# Discrétiser les processus de Hawkes
val_discret_simulated_events_seqs = discretise(val_simulated_events_seqs, filename='val_binned_hawkes_simulations.csv')

In [ ]:
# Testing dataset generation

# Intensity Decay Parameter (β) = U(p = 1, q = 3)
# Branching Ratio (η) = U(a = 0.05, b = 0.8)
# Expected Activity (E) = 500
# Time Horizon (T) = 100
# Interval Length (∆) = 1
# Testing processes = 20_000

# Hawkes process hyper-parameters generation
test_params, test_alpha, test_beta, test_mu = hyper_params_simulation(filename="test_hawkes_hyperparams.csv")
# Hawkes processes simulations
test_simulated_events_seqs = hawkes_simulations(test_mu, test_alpha, test_beta, filename='test_hawkes_simulations.csv')
# Discrétiser les processus de Hawkes
test_discret_simulated_events_seqs = discretise(test_simulated_events_seqs, filename='test_binned_hawkes_simulations.csv')

In [149]:
from functools import partial
from UTILS.utils import write_csv

def discretise(jump_times: np.ndarray, filename: str = 'binned_hawkes_simulations.csv') -> np.ndarray:

    # Computed bins number
    num_bins = int(var.TIME_HORIZON // var.DISCRETISE_STEP)

    # For each process (j), compute jump times histogram (h) using the intervals boundaries specified by the bins
    counts = np.array(list(map(partial(lambda h: np.histogram(h, bins=np.linspace(0, var.TIME_HORIZON, num_bins + 1))[0]), jump_times)), dtype=np.float64)
                                    
    # Created ist of dictionaries representing the binned simulated event sequences
    counts_list = list(map(partial(lambda _, row: {str(idx): x for idx, x in enumerate(row)}, range(var.TIME_HORIZON)), counts))

    # Write the counts to a CSV file
    write_csv(counts_list, filename=filename)

    return counts
    

def discretise2(jump_times: np.ndarray, filename: str = 'binned_hawkes_simulations.csv') -> np.ndarray:

    # Computed bins number
    num_bins = int(var.TIME_HORIZON // var.DISCRETISE_STEP)

    # Initialized an array of zeros with dimensions (number of processes, number of jumps per unit of time)
    counts = np.zeros((len(jump_times), num_bins), dtype=np.float64)

    # For each process (j), compute jump times histogram (h) using the intervals boundaries specified by the bins
    for j, h in enumerate(jump_times):
        counts[j], _ = np.histogram(h, bins=np.linspace(0, var.TIME_HORIZON, num_bins + 1))
                                    
    # Created ist of dictionaries representing the binned simulated event sequences
    counts_list = list(map(partial(lambda _, row: {str(idx): x for idx, x in enumerate(row)}, range(var.TIME_HORIZON)), counts))

    # Write the counts to a CSV file
    write_csv(counts_list, filename=filename)

    return counts

%timeit -r 100 -n 1000 discretise(train_simulated_events_seqs)
%timeit -r 100 -n 1000 discretise2(train_simulated_events_seqs)

4.11 ms ± 276 µs per loop (mean ± std. dev. of 100 runs, 1,000 loops each)
4.08 ms ± 245 µs per loop (mean ± std. dev. of 100 runs, 1,000 loops each)
